In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten,GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import scipy
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
print("Num GPUs Available: ", len(tf.config.list_physical_devices()))
tf.config.list_physical_devices()

Num GPUs Available:  2


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
src_path_train = "/content/drive/MyDrive/faces/train"
src_path_val = "/content/drive/MyDrive/faces/val"

num_classes = len(os.listdir("/content/drive/MyDrive/faces/train"))

train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory=src_path_train,
    target_size=(224,224),
    color_mode="rgb",
    class_mode="sparse",
    subset='training',
    shuffle=True,
    seed=42,
    batch_size=4)

valid_generator = val_datagen.flow_from_directory(
    directory=src_path_val,
    target_size=(224,224),
    color_mode="rgb",
    batch_size=2,
    class_mode="sparse",
    shuffle=True,
    seed=42
)


Found 1428 images belonging to 46 classes.
Found 502 images belonging to 46 classes.


In [ ]:
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['sgd']))
HP_LEARNING_RATE= hp.HParam('learning_rate', hp.Discrete([0.0001]))
METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_LEARNING_RATE,HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [ ]:

def train_test_model(hparams,model_name):
    base_model = tf.keras.applications.ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet")
    
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(hparams[HP_DROPOUT])(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )
    checkpoint = ModelCheckpoint("models/"+model_name+"_best.h5", monitor='val_accuracy',verbose=2, save_best_only=True, mode='max')
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=25, verbose=1, mode='min')
    model.fit(train_generator, validation_data= valid_generator,epochs=40,callbacks=[checkpoint,early,tf.keras.callbacks.TensorBoard("logs/"),  # log metrics
        hp.KerasCallback("logs/", hparams)],verbose = 1)

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)
    
    model_name = run_dir.split("/")[-1]
    print("model name: "+ model_name)
    train_test_model(hparams, model_name)


In [ ]:
session_num = 0

for lr in HP_LEARNING_RATE.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.values):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_LEARNING_RATE: lr,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run_%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

--- Starting trial: run_0
{'learning_rate': 0.0001, 'dropout': 0.1, 'optimizer': 'sgd'}
model name: run_0
Epoch 1/40
357/357 [==============================] - ETA: 0s - loss: 249.8736 - accuracy: 0.1036
Epoch 1: val_accuracy improved from -inf to 0.01394, saving model to /content/drive/MyDrive/faces/run_0_best.h5
357/357 [==============================] - 46s 119ms/step - loss: 249.8736 - accuracy: 0.1036 - val_loss: 134.6288 - val_accuracy: 0.0139
Epoch 2/40
357/357 [==============================] - ETA: 0s - loss: 54.5423 - accuracy: 0.2983
Epoch 2: val_accuracy improved from 0.01394 to 0.03187, saving model to /content/drive/MyDrive/faces/run_0_best.h5
357/357 [==============================] - 45s 125ms/step - loss: 54.5423 - accuracy: 0.2983 - val_loss: 16.1470 - val_accuracy: 0.0319
Epoch 3/40
357/357 [==============================] - ETA: 0s - loss: 8.8421 - accuracy: 0.4972
Epoch 3: val_accuracy improved from 0.03187 to 0.27689, saving model to /content/drive/MyDrive/faces/r